## Convergence Bispectrum Calculations

In [63]:
import numpy as np
import healpy as hp
import bispec_calculator as bispec
from helper_funcs import *

The goal of this Jupyter notebook is to calculate the Theoretical Convergence Bispectrum, which is given by $\texttt{Munshi et. al}$'s paper as the equation

$$B_{\ell_1, \ell_2, \ell_3} = \int_0^{\chi_s} d\chi \left[\frac{3\Omega_M H_0^2}{2a(\chi)}\right] \chi^2 W^3(\chi, \chi_s) B_{\delta}\left(\frac{\ell_1}{\chi}, \frac{\ell_2}{\chi}, \frac{\ell_3}{\chi}, \chi\right)$$

Since the source plan is just the CMB plane here, then we can find the comoving distance by converting the redshift of the CMB plane to comoving distance:

In [64]:
from scipy.integrate import quad
from scipy import interpolate
from astropy.cosmology import Planck18 as cosmo
from astropy.constants import c
from astropy import units as u

def interp_chi_and_z(z: int) -> float:
    """
    Interpolate comoving and redshift

    Parameters:
        z (int) : redshift
    
    Returns:
        chi_from_z (float) : comoving distance
    """
    c_km = c.to(u.km/u.s).value

    def _invH(z):
        return c_km/cosmo.H(z).value

    zs = np.logspace(-2, 3, 1000)
    dcm = np.zeros_like(zs)

    for i,z in enumerate(zs):
        dcm[i] = quad(_invH, 0, z)[0]

    chi = interpolate.interp1d(zs, dcm, bounds_error=False, fill_value = 0)
    z = interpolate.interp1d(dcm, zs, bounds_error=False, fill_value = 0)

    return chi, z

In [65]:
def compute_scale_factor(z: int) -> float:
    """
    Finds the scale factor a(z) from z.

    Parameters:
        z (int) : redshift

    Returns:
        a (float) : scale factor
    """
    return 1 / (1 + z)

Create $P_k$ as a function of $k$ and $z$ and create a 2-d interpolation.

To Do:
- Check how long `create_pk` takes to call.
- CAMB Matter Power Interpolator
    - Check to make sure that I'm putting the right Cosmology in
    - Check units after?
    - Based on RectBivariateSpline

*Dimensionaless vs. Dimensional $P(k)$ units, check to make sure that these factors are/aren't there before integration*

Div0 Question:
- Start from 0.01

In [66]:
from camb import get_matter_power_interpolator, model


# zmin – minimum z (use 0 or smaller than you want for good interpolation)

# zmax – maximum z (use larger than you want for good interpolation)

# nz_step – number of steps to sample in z (default max allowed is 100)

# Using Planck 2018 parameters
def create_pk(kmax, zmax, zmin=0, nz_step=20, nonlinear=False):
    """
    Creates Power Spectrum at a certain redshift based on Planck 2018 parameters

    Parameters:
        kmax (ndarray) : ks
        zs (ndarray) : zs
        
    returns:
        P_k interpolator : matter power spectrum interpolator
    """

    # Cosmological parameters for Planck18
    Omega_c = 0.26067
    Omega_b = 0.049
    h = 0.6766
    n_s = 0.9665
    sigma8 = 0.8102
    transfer_function = 'boltzmann_camb'

    cosmo_model = model.CAMBparams()
    cosmo_model.set_cosmology(H0=100*h, ombh2=Omega_b, omch2=Omega_c)

    Pk_interpolator = get_matter_power_interpolator(params=cosmo_model,
                                                    zmin=zmin,
                                                    zmax=zmax,
                                                    nz_step=nz_step,
                                                    kmax=kmax,
                                                    hubble_units=False,
                                                    nonlinear=nonlinear)
                                                    
    return Pk_interpolator

reasonable value for kmax \in [1,10]

reasonable for zmax is source redshift --> if we're using z=2, set to 2.5 so we don't have any interpolation issues

In [67]:
zmax = 2.5
kmax = 10

P_k = create_pk(zmax, kmax)

First thing we need to do is calculate $B_{\delta}(\mathbf{k}_1, \mathbf{k}_2, \mathbf{k}_3; \chi)$, which is given by:

$$B_{\delta}(\mathbf{k}_1, \mathbf{k}_2, \mathbf{k}_3; \chi) = 2F_2(\mathbf{k}_1, \mathbf{k}_2, z)P_{\delta}(\mathbf{k}_1,z)P_{\delta}(\mathbf{k_2},z) + \text{cyc. perm}$$

such that:

$$F_2(\mathbf{k}_1, \mathbf{k}_2, z) = \frac57 a(k_1, z)a(k_2, z) + \frac12 \frac{k_1^2 + k_2^2}{k_1k_2}b(k_1, z)b (k_2,z)\cos\theta + \frac27 c(k_1, z)c(k_2,z)$$

where we're using $a = b = c = 1$, giving us:

$$F_2(\mathbf{k}_1, \mathbf{k}_2, z) = \frac57 + \frac12 \frac{k_1^2 + k_2^2}{k_1k_2}\cos\theta + \frac27$$

We'll need this for every cyclic permutation, so let's code up a general function to solve for $F_2$:

Now let's find the overall expression for $B_{\delta}(\mathbf{k}_1, \mathbf{k}_2, \mathbf{k}_3; \chi)$:

In [68]:
# F2 function
def F2(k1, k2):
    return 5/7 + 1/2 * (k1**2 + k2**2)/(k1 * k2) * np.cos(np.dot(k1, k2)) + 2/7

def create_B_delta_term(P_k, k_i, k_j, z):
    return 2 * F2(k_i, k_j) * P_k(k_i, z) * P_k(k_j, z)

def compute_B_delta(chi, P_k, k1, k2, k3, z_int):
    ks_i = [k1, k2, k3]
    ks_j = [k2, k3, k1]
    
    B_delta = 0
    for k_i, k_j in zip(ks_i, ks_j):
        B_delta += create_B_delta_term(P_k, k_i, k_j, z_int(chi))
    
    return B_delta

Now let's find the lensing kernel, which is given by:

$$W(\chi, \chi_s) = \frac{\chi_s - \chi}{\chi\chi_s}\Theta(\chi_s - \chi)$$

where the step function $\Theta(\chi_s - \chi)$ is there to ensure that the lensing can’t have a contribution from before the source existed. We can control this via integration, so we'll let $\Theta(\chi_s - \chi) = 1$, leaving us with:

$$W(\chi, \chi_s) = \frac{\chi_s - \chi}{\chi\chi_s}$$

In [69]:
def lensing_function(chi, chi_s):
    """
    Find the lensing function W(chi, chi_s)
    """
    return (chi_s - chi) / (chi * chi_s)

The redshift of the CMB is apparently $z_{CMB} = 1089$, so:

Combining everything together:

$$B_{\ell_1, \ell_2, \ell_3} = \int_0^{\chi_s} d\chi \left[\frac{3\Omega_M H_0^2}{2a(\chi)}\right] \chi^2 W^3(\chi, \chi_s) B_{\delta}\left(\frac{\ell_1}{\chi}, \frac{\ell_2}{\chi}, \frac{\ell_3}{\chi}, \chi\right)$$

In [70]:
def _convergence_integrand(chi, P_k, l1, l2, l3, z_int, chi_s):
    """
    Convergence integrand
    """
    B_delta = compute_B_delta(chi, P_k, l1/chi, l2/chi, l3/chi, z_int)

    W_chi = lensing_function(chi, chi_s)

    return chi**2 * W_chi**3 * B_delta

In [73]:
from scipy.integrate import quad

# def integrand(integration_variable, b, c ,d ,e ,f)
# romberg( integrand, 0, 1, args= (b,c,d,e,f))

def compute_convergence_bispec(l1, l2, l3, lb=1, z_s=1089):
    """
    Compute convergence bispec. Will finish later.
    """
    Omega_m = cosmo.Om(zmax) # this is dependent on z so also integral?
    H0 = cosmo.H0.value

    coefficient = (3 * Omega_m * H0**2)/(2 * compute_scale_factor(zmax))

    lmax = max(l1, l2, l3)
    kmax = lmax / lb
    P_k = create_pk(zmax, kmax)
    z_int = interp_chi_and_z(zmax)[1]

    chi_s = z_int(z_s)
    bispec = quad(_convergence_integrand, lb, chi_s, args=(P_k, l1, l2, l3, z_int, chi_s))[0]

    return coefficient * bispec

In [82]:
def test_time():
    for z in range(1, 25):
        create_pk(z, 10)
        interp_chi_and_z(z)[0]

In [83]:
%timeit test_time

89.5 ns ± 2.15 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [74]:
l = 10

bl = compute_convergence_bispec(l, l, l)

2210488.8961837045


/tmp/ipykernel_40143/387298328.py:21: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  bispec = quad(_convergence_integrand, lb, chi_s, args=(P_k, l1, l2, l3, z_int, chi_s))[0]


Next step:

- Diagram --> code being repeated (helper funcs?), take things that can be passed in as an argument outside to the wrapping loop function
- Write a wrapping loop function to compute bls across range of $\ell$ values

In [78]:
# from helper_funcs import *

# ks = np.logspace(-3, 1, 100)
# plot_pk(ks, P_k(0, ks)[0])

In [76]:
def plot_bispec_eq(bls, ells,
                   figsize=(12,6), 
                   lmin=0, xlmax=None,
                   xscale='linear', yscale='linear',
                   xlabel="even multipole triplet $(\ell_1, \ell_2, \ell_3)$",
                   ylabel="$B_{(\ell_1, \ell_2, \ell_3)}$",
                   title="Bispectrum", scheme='e'):
    if scheme == 'e':
        plt.figure(figsize=figsize)
        plt.plot(ells, bls)
        plt.legend(frameon=False, loc='upper right', fontsize=14)
        plt.xscale(xscale)
        plt.yscale(yscale)
        plt.ylabel(ylabel, fontsize=18)
        plt.xlabel(xlabel, fontsize=18)
        plt.title(title, fontsize=18)
        plt.xticks(fontsize=18)
        plt.yticks(fontsize=18)
        plt.show()